# Testing Dask

In [6]:
from code.util import *
from code.preprocessor import *
from code.gofaster import *
import dask.dataframe as dd
import pandas as pd

gf = GoFaster(12, 12)

In [2]:
pframe = load("data/urls.pkl")

In [3]:
dframe = dd.from_pandas(pframe, npartitions=4)

In [4]:
def worker(row):
    return tokenize(row.url)

In [5]:
def gf_worker(df):
    df["tokens"] = df.apply(worker, axis=1)
    return df

In [ ]:
%%time
pframe["tokens"] = pframe.apply(worker, axis=1)

In [7]:
%%time
pframe = gf.parallelize(pframe, gf_worker)

CPU times: user 2.31 s, sys: 614 ms, total: 2.92 s
Wall time: 6.79 s


In [ ]:
%%time
dframe["tokens"] = dframe.apply(worker, axis=1)
dframe = dframe.compute()

In [ ]:
dframe.head()

# Keras Text Tokenizer

In [1]:
import plaidml.keras
plaidml.keras.install_backend()

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import lightgbm
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from code.util import *

In [2]:
urls = load("data/urls_sampled.pkl")

In [49]:
x = urls.url
y = urls.target

In [50]:
tk = Tokenizer(
    num_words=6000,
    filters='!"#$%&()*+,-.:;<=>?@[\\]^_`{|}~',
    split="/"
)

In [82]:
tkc = Tokenizer(
    filters="",
    char_level=True
)

In [51]:
tk.fit_on_texts(x)

In [83]:
tkc.fit_on_texts(x)

In [33]:
seq = tk.texts_to_sequences(x)

In [84]:
seqc = tkc.texts_to_sequences(x)

In [58]:
x0 = pad_sequences(seq, maxlen=20)

In [85]:
x0 = pad_sequences(seqc, maxlen=128)

In [86]:
x_train, x_test, y_train, y_test = train_test_split(x0, y, test_size=0.3, random_state=42)

In [91]:
lgbm = lightgbm.LGBMClassifier(
    n_estimators=1500,
    max_depth=-1,
    objective="binary",
    subsample_for_bin=2000,
    n_jobs=6,
    random_state=42
)

In [92]:
lgbm.fit(x_train, y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=1500, n_jobs=6, num_leaves=31, objective='binary',
               random_state=42, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=2000, subsample_freq=0)

In [93]:
y_pred = lgbm.predict(x_test)

In [94]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.96      0.99      0.97     31562
           1       0.93      0.81      0.87      6624

    accuracy                           0.96     38186
   macro avg       0.95      0.90      0.92     38186
weighted avg       0.96      0.96      0.96     38186



In [100]:
len(y_test[y_test+y_pred==1])

1655